In [ ]:
import sys
sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy


## Read the QB's sample

In [ ]:
# Read preprocessed data
df = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
# df = df.query("topic_label_de!='unbekannt'").reset_index()
df = df.drop(columns=['docid'])
df.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
df['docid'] = df.index

df['content'] = df['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

df['content'] = df['content'].apply(lambda x : preprocess_documents(x, tagger_de))
df

## Calculate Embeddings and Initialize NMSLIB Indices

In [ ]:
model_1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
model_2 = SentenceTransformer("aari1995/German_Semantic_STS_V2")
# model_2.tokenizer.pad_token = model_2.tokenizer.eos_token

model_3 = SentenceTransformer("sentence-transformers/LaBSE")
model_4 = SentenceTransformer("PM-AI/bi-encoder_msmarco_bert-base_german")
model_5 = SentenceTransformer("efederici/e5-base-multilingual-4096")
model_6 = SentenceTransformer("intfloat/multilingual-e5-base")
model_7 = SentenceTransformer("clips/mfaq")
model_8 = SentenceTransformer("PM-AI/sts_paraphrase_xlm-roberta-base_de-en")
model_9 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-euclidean")
model_10 = SentenceTransformer("LLukas22/all-MiniLM-L12-v2-embedding-all")
model_11 = SentenceTransformer("LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all")
model_12 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")
model_13 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
model_14 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-cosine")
model_15 = SentenceTransformer("shibing624/text2vec-base-multilingual")
model_16 = SentenceTransformer("Sahajtomar/German-semantic")
model_17 = SentenceTransformer("setu4993/LaBSE")
model_18 = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
model_19 = SentenceTransformer("and-effect/musterdatenkatalog_clf")
model_20 = SentenceTransformer("nblokker/debatenet-2-cat")
model_21 = SentenceTransformer("setu4993/LEALLA-large")
model_22 = SentenceTransformer("dell-research-harvard/lt-wikidata-comp-de")
model_23 = SentenceTransformer("ef-zulla/e5-multi-sml-torch")
model_24 = SentenceTransformer("barisaydin/text2vec-base-multilingual")
model_25 = SentenceTransformer("meta-llama/Llama-2-7b-chat-hf")
model_25.tokenizer.pad_token = model_25.tokenizer.eos_token


In [ ]:
# # Create the indices

index_1 = create_index(df, model_1, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
index_2 = create_index(df, model_2, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/aari1995-German_Semantic_STS_V2.nmslib")
index_3 = create_index(df, model_3, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-LaBSE.nmslib")
index_4 = create_index(df, model_4, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
index_5 = create_index(df, model_5, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/efederici-e5-base-multilingual-4096.nmslib")
index_6 = create_index(df, model_6, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/intfloat-multilingual-e5-base.nmslib")
index_7 = create_index(df, model_7, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/clips-mfaq.nmslib")
index_8 = create_index(df, model_8, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
index_9 = create_index(df, model_9, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
index_10 = create_index(df, model_10, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
index_11 = create_index(df, model_11, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
index_12 = create_index(df, model_12, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
index_13 = create_index(df, model_13, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
index_14 = create_index(df, model_14, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
index_15 = create_index(df, model_15, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/shibing624-text2vec-base-multilingual.nmslib")
index_16 = create_index(df, model_16, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/Sahajtomar-German-semantic.nmslib")
index_17 = create_index(df, model_17, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/setu4993-LaBSE.nmslib")
index_18 = create_index(df, model_18, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
index_19 = create_index(df, model_19, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/and-effect-musterdatenkatalog_clf.nmslib")
index_20 = create_index(df, model_20, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/nblokker-debatenet-2-cat.nmslib")
index_21 = create_index(df, model_21, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/setu4993-LEALLA-large.nmslib")
index_22 = create_index(df, model_22, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
index_23 = create_index(df, model_23, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/ef-zulla-e5-multi-sml-torch.nmslib")
index_24 = create_index(df, model_24, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/barisaydin-text2vec-base-multilingual.nmslib")
index_25 = create_index(df, model_25, index_path="/Users/user/question-retrieval-KIPerWeb/pool_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


# # Or load them:

# index_1 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# index_2 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/aari1995-German_Semantic_STS_V2.nmslib")
# index_3 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-LaBSE.nmslib")
# index_4 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# index_5 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/efederici-e5-base-multilingual-4096.nmslib")
# index_6 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/intfloat-multilingual-e5-base.nmslib")
# index_7 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/clips-mfaq.nmslib")
# index_8 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# index_9 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# index_10 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# index_11 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# index_12 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# index_13 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# index_14 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# index_15 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/shibing624-text2vec-base-multilingual.nmslib")
# index_16 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/Sahajtomar-German-semantic.nmslib")
# index_17 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/setu4993-LaBSE.nmslib")
# index_18 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# index_19 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/and-effect-musterdatenkatalog_clf.nmslib")
# index_20 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/nblokker-debatenet-2-cat.nmslib")
# index_21 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/setu4993-LEALLA-large.nmslib")
# index_22 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# index_23 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/ef-zulla-e5-multi-sml-torch.nmslib")
# index_24 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/barisaydin-text2vec-base-multilingual.nmslib")
# # index_25 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/pool_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


## Run Queries Against Search Indices

In [ ]:
query_test = search('multikulturelle kompetenz'.lower(), index_2, model_2, df, k=100)['search_output']
print(len(query_test))
query_test

In [ ]:
## We could use queries:
# queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/topics_experiments/topics/topics.csv")
queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/queries/queries.csv")
queries_file['queries'] = queries_file['queries'].apply(clean_text)



print("-> Number of queries:",len(queries_file['queries'].tolist()))

queries = queries_file['queries'].tolist()
query_ids = queries_file['qid'].tolist()
queries_file

## Generate searches and store them as JSON files

In [ ]:
path = '/Users/user/question-retrieval-KIPerWeb/runs/pool/json_format/'


progress_bar = tqdm(zip(queries, query_ids), total=len(queries), desc="Processing")

for i, j in progress_bar:
    progress_bar.set_description(f"Processing query: {i}")
    search(i, index_1, model_1, df, k=100, run_path=f'{path}query_{j}/run_01.json')
    search(i, index_2, model_2, df, k=100, run_path=f'{path}query_{j}/run_02.json')
    search(i, index_3, model_3, df, k=100, run_path=f'{path}query_{j}/run_03.json')
    search(i, index_4, model_4, df, k=100, run_path=f'{path}query_{j}/run_04.json')
    search(i, index_5, model_5, df, k=100, run_path=f'{path}query_{j}/run_05.json')
    search(i, index_6, model_6, df, k=100, run_path=f'{path}query_{j}/run_06.json')
    search(i, index_7, model_7, df, k=100, run_path=f'{path}query_{j}/run_07.json')
    search(i, index_8, model_8, df, k=100, run_path=f'{path}query_{j}/run_08.json')
    search(i, index_9, model_9, df, k=100, run_path=f'{path}query_{j}/run_09.json')
    search(i, index_10, model_10, df, k=100, run_path=f'{path}query_{j}/run_10.json')
    search(i, index_11, model_11, df, k=100, run_path=f'{path}query_{j}/run_11.json')
    search(i, index_12, model_12, df, k=100, run_path=f'{path}query_{j}/run_12.json')
    search(i, index_13, model_13, df, k=100, run_path=f'{path}query_{j}/run_13.json')
    search(i, index_14, model_14, df, k=100, run_path=f'{path}query_{j}/run_14.json')
    search(i, index_15, model_15, df, k=100, run_path=f'{path}query_{j}/run_15.json')
    search(i, index_16, model_16, df, k=100, run_path=f'{path}query_{j}/run_16.json')
    search(i, index_17, model_17, df, k=100, run_path=f'{path}query_{j}/run_17.json')
    search(i, index_18, model_18, df, k=100, run_path=f'{path}query_{j}/run_18.json')
    search(i, index_19, model_19, df, k=100, run_path=f'{path}query_{j}/run_19.json')
    search(i, index_20, model_20, df, k=100, run_path=f'{path}query_{j}/run_20.json')
    search(i, index_21, model_21, df, k=100, run_path=f'{path}query_{j}/run_21.json')
    search(i, index_22, model_22, df, k=100, run_path=f'{path}query_{j}/run_22.json')
    search(i, index_23, model_23, df, k=100, run_path=f'{path}query_{j}/run_23.json')
    search(i, index_24, model_24, df, k=100, run_path=f'{path}query_{j}/run_24.json')
    search(i, index_25, model_25, df, k=100, run_path=f'{path}query_{j}/run_25.json')

## Preprocess all the synthetic searches and turn them into TREC style files

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/pool/json_format/'
json_files = list_files_from_dir(path, extention='.json')
json_files



In [ ]:
# For each JSON output, transform the results to a TREC format

for e in json_files:
    try:
        to_trec_format(e)
    except IndexError:
        print(f"Error! in {e}")

In [ ]:
#list the TREC formatted files, these have a .tsv extention
list_files_from_dir(path, extention='.tsv')

# Process all the runs and continue with the trec formatting
path_list_processed = list_files_from_dir(path, extention='.tsv')

list_of_dfs = []
for e in path_list_processed:
    df = pd.concat([pd.read_csv(e, header=None, sep='\t', names=["queries", "Q0", "docno", "rank", "score", "tag"])])
    list_of_dfs.append(df)
    

all_runs = pd.concat(list_of_dfs).reset_index(drop=True)


all_runs= all_runs.merge(queries_file,on='queries')
all_runs = all_runs[["qid", "Q0", "docno", "rank", "score", "tag"]]
# all_runs=all_runs.rename({"query_id":"qid"}, axis='columns')

# dict(all_runs['qid'].value_counts())
all_runs

# queries_file

In [ ]:
# Extract runs by different LMs
queries = list(set(all_runs['qid']))
# queries = [e.split()[0] for e in queries]
# print(queries)
L_models = list(set(all_runs['tag']))
runs = [all_runs.query(f'tag=="{e}"') for e in L_models]

for i, j in zip(runs, L_models):
    i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/pool/trec_format/{j}_run.txt', header = None, sep='\t',index=False )
    

## Create a pool with all the search outputs

In [ ]:

trec_format_files = list_files_from_dir("/Users/user/question-retrieval-KIPerWeb/runs/pool/trec_format/", extention = '.txt')

runs_list = []
for e in trec_format_files:
    runs_list.append(TrecRun(e))
    

pool1 = TrecPoolMaker().make_pool(runs_list, strategy="topX", topX=100)
pool1

In [ ]:
# print("coverage")
# for e in runs_list:
#     print(f" {str(e).split()[-1]} {pool1.check_coverage(e, topX=100)}")

In [ ]:
pool_path = "/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/raw_pool.txt"
pool1.export_document_list(filename = pool_path, with_format="relevation")


In [ ]:
pool = pd.read_csv(pool_path, sep='\t', header=None, names=['qid', 'Q0', 'docid', 'col1', 'col2', 'format'])
pool

In [ ]:
# Read preprocessed data
df = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
# df = df.query("topic_label_de!='unbekannt'").reset_index()
df = df.drop(columns=['docid'])
df.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
df['docid'] = df.index

df['content'] = df['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

df['content'] = df['content'].apply(lambda x : preprocess_documents(x, tagger_de))


In [ ]:

q_bank = df.drop(columns=['docid'])
q_bank['docid'] = df.index

pool = q_bank.merge(pool, on='docid')[['content', 'docid', 'qid', 'Q0', 'col1', 'col2', 'format','topic_label_de_fixed']]
pool


In [ ]:
pool_w_info = pool.merge(queries_file, on='qid')
pool_w_info = pool_w_info.rename({"queries":"query", "topic_label_de_fixed": "category"}, axis='columns')
pool_w_info = pool_w_info[["content", "query", "category", "docid", "qid", "Q0", "col1", "col2", "format"]]
pool_w_info

In [ ]:
pool_w_info.to_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/pool_w_metadata.txt", 
                      index=False)


In [ ]:
import pandas as pd

df = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/pool_w_metadata.txt")
dict(df['query'].value_counts(''))